### Gradient Descent Template

In [ ]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [7]:
def f(x):
    return np.sum((sigmoid(A.dot(x))-b)**2)

In [8]:
# at zero vector, on this particular set of test data, f should give 2.
# run the next line to make sure it does
x = np.zeros((d,1))
assert f(x) == 2.0

In [9]:
# Normally, you should keep our test code. 
# For various reasons, == should actually work here. 
# But, we allow you to replace this condition with something like
assert np.abs(f(x)-2.0) < 10e-14

In [10]:
def sigmoid_deriv(z):
    return 1./(np.exp(-z/2)+np.exp(z/2))**2

In [11]:
# check that it is equal = 0.25
sigmoid_deriv(0)

0.25

### check it at a random point.  
Confirm that the values displayed on the next two lines are close.

In [12]:
x = np.random.randn(1)
delta = .0001
(sigmoid(x+delta)-sigmoid(x-delta))/(2*delta)

array([0.13425617])

In [13]:
sigmoid_deriv(x)

array([0.13425617])

### Make sure that when you apply sigmoid_deriv to a vector, you get the vector of derivatives

In [14]:
x = np.array([0, 1, 2])
print(sigmoid_deriv(x))

[0.25       0.19661193 0.10499359]


In [15]:
assert x.shape == sigmoid_deriv(x).shape

### create a function that computes the gradient of f.  Call it grad_f.
Put it in the following cell

We recall that the gradient of objective is
$$
\nabla f(x) = 2 \sum_{i=1}^n (\sigma(a_i^T x) - b_i)\sigma^{'}(a_i^T x) a_i
$$

Also recall that $a_i^T$ are the rows of the matrix $A$. If we let $\epsilon_i = (\sigma(a_i^T x) - b_i)\sigma^{'}(a_i^T x)$, then we can simplify the above to
$$
2 A^T \epsilon
$$

In [16]:
def grad_f(x):
    e=(sigmoid(A.dot(x))-b)*sigmoid_deriv(A.dot(x))
    return 2*A.T.dot(e)

### Check that your computation is correct on random vectors.
The numbers printed in the next line should be close to each other.

In [17]:
x0 = np.random.randn(d,1)
r = np.random.randn(d,1)/10**6
print(f(x0+r) - f(x0))
print(grad_f(x0).T @ (r) )

-3.825378298571991e-07
[[-3.82537776e-07]]


Or write a test like the one we did in class

### Simulate the data

In [18]:
hash_object = hashlib.md5(netid.encode('utf-8'))
hex_hash = hash_object.hexdigest()
seed = int(hex_hash[0:4],16)
print("seed : ", seed)
np.random.seed(int(hex_hash[0:4],16))

seed :  62817


In [19]:
n = 30
d = 10

A = np.random.randn(n,d)
b = np.round(np.random.rand(n,1))

### Run the following cell.  It should not produce an error.
We will want to see the results.

In [20]:
assert A.shape == (30,10)
print(np.linalg.norm(A))

16.184831753877276


## 5. Gradient Descent Optimization
Write some code to do gradient descent.
Find a vector xx for which the norm of the gradient of f at xx is at most 10^(-6).

**Please implement step size control**

In [21]:

def grad_descent(x0, f, g, tol=10**(-6),step_size=50,max_iters=100000):

    f0 = f(x0)  # initial value
    g0 = g(x0)  # initial gradient
    
    dbg_iters_actual  =0               # I'm going to use this to log the actual number of tryes I had, including things internal step size control
    dbg_step_cond_met =0               # debugging information I like to collect.
    for iters in range(max_iters):     # I only use for loop to avoid getting stuck somewhere. 
        if np.linalg.norm(g0)<tol:     # Stop condition based on gradient norm as I was asked to do. I would often check how much progress I made in recent iterations.
            break
        if iters%10000 ==0:             # Once a while print some information. I like to do this when I tests.             
            # I do this at each iteration when I develop the code.
            # Normally I would disable this in code I give out, but I would usually keep something in code I work with            
            #f0 = f(x0)
            #g0 = g(x0)
            print("iteration information: ",iters , dbg_iters_actual, f0, np.linalg.norm(g0)) 
        tstep_size = step_size    # initialize step size to the same initial value at each iteration        
        dbg_step_cond_met = -1    # I want to know if the step size worked.
        for j1 in range(200):     # Step size control iterations. Try smaller steps until we get this right.
            dbg_iters_actual = dbg_iters_actual+1
            xnew = x0-tstep_size*g0
            fnew = f(xnew)
            if (j1>10):            
                # I really want to know what's happening here. 
                # Again, when I develop I print everyhitng!
                # I do this at each iteration when I develop the code.
                # Normally I would disable this in code I give out, but I would usually keep something in code I work with            
                print("     step size control information: ",iters , dbg_iters_actual, "\t", f0, np.linalg.norm(g0), j1, tstep_size, fnew)             
            if (fnew < f0): # If the step improves things
                dbg_step_cond_met = j1  # Ok, this worked at the j1-th try
                f0 = fnew   # the best function value
                x0 = xnew   # the corresponding vector
                g0 = g(x0)  # and, its gradient
                break       # Stop if we hit the right point
            else:           # try again with smaller step
                tstep_size = tstep_size/2.0
                continue
            print("ERROR: step control is stuck")  # Ha. We're not supposed to be in this state!
        if dbg_step_cond_met<0:                    # Ha. We're not supposed to be in this state!
            print("ERROR: step control didn't work, step still too large?")  
    
    print("done:", iters, dbg_iters_actual, "\t loss:", f(x0), "\t   grad norm:", np.linalg.norm(g(x0)))
    return x0

In [22]:
np.random.seed(123)
xxi = np.random.randn(d,1)
t0=time.time()
xx = grad_descent(xxi, f, grad_f, tol=10**(-6),step_size=100,max_iters=100)
t1=time.time()
print("time:",t1-t0)
print("loss:", f(xx))

iteration information:  0 0 12.247870539709655 2.3767478991885196
done: 2 2 	 loss: 7.0 	   grad norm: 5.500770241481688e-17
time: 0.0010654926300048828
loss: 7.0


Check that yout solutions satifies the requirement for xx

In [23]:
if np.linalg.norm(grad_f(xx)) < 1e-6:
    print("gradient at xx is small!")
else:
    print ("xx failed gradient test")

gradient at xx is small!


What is the loss for your solution?

In [24]:
print(f(xx))

7.0


Now, try to run your optimization with a different starting point, and call your new solution xx2.
Does it satisfy the requirement above? What is the loss for that solution?

In [25]:
np.random.seed(1)
xx2i = np.random.randn(d,1)
t0=time.time()
xx2 = grad_descent(xx2i, f, grad_f, tol=10**(-6),step_size=100,max_iters=200000)
t1=time.time()
print("time:",t1-t0)
print("loss:", f(xx2))
# Intereting, this point happens to take much longer, but loss happens to be much better!

iteration information:  0 0 10.198744507199393 1.3815220545796596
iteration information:  10000 12276 1.0000296729449263 5.593211668198365e-06
iteration information:  20000 22276 1.0000144388213843 2.723290525486981e-06
iteration information:  30000 32276 1.0000095381621974 1.799450329931623e-06
iteration information:  40000 42276 1.0000071204678045 1.343539172137822e-06
iteration information:  50000 52276 1.0000056802920478 1.071908476478321e-06
done: 53557 55833 	 loss: 1.000005299014613 	   grad norm: 9.999886444326768e-07
time: 2.9832687377929688
loss: 1.000005299014613


### Check the problem is convex or not

Note that the function we study here is NOT exactly the logistic regression. We know that logistic regression is convex, but we don't know if this problem is convex. We will try to show next that it is not!

In [26]:
# hmmm.... unless I'm lucky, trial and error is a lot of work... I'll make the computer do it instead...
np.random.seed(123)
dbg_founnd = -1
for jtrial in range(1000):
    print("\n\n=== TRIAL ",jtrial)
    t0=time.time()
    x1i = np.random.randn(d,1)
    x2i = np.random.randn(d,1)
    x1 = grad_descent(x1i, f, grad_f, tol=10**(-6),step_size=50,max_iters=200000)
    x2 = grad_descent(x2i, f, grad_f, tol=10**(-6),step_size=50,max_iters=200000)
    t1=time.time()
    print("time:",t1-t0)
    print("losses:", f(x1), f(x2))
    print("grads:", np.linalg.norm(grad_f(x1)), np.linalg.norm(grad_f(x2)))
    print("convexity test:", f((x1+x2)/2) , max(f(x1),f(x2)) )
    
    # I
    if np.linalg.norm(grad_f(x1)) > 1e-6:
        print ("x1 failed gradient test")
        continue        
    else:
        print("gradient at x1 is small!")
        if np.linalg.norm(grad_f(x2)) > 1e-6:
            print ("x2 failed gradient test")
            continue
        else:
            print ("gradient at x2 is small!")
            if f((x1+x2)/2) - max(f(x1),f(x2)) <= 0.1:
                print ("The value between is not large enough. Find different points.")            
                continue
            else:
                print ("The value between is significantly higher! Sucess!")
                dbg_founnd = jtrial
                break




=== TRIAL  0
iteration information:  0 0 12.247870539709655 2.3767478991885196
done: 7070 7072 	 loss: 7.000000359167062 	   grad norm: 9.99997276816361e-07
iteration information:  0 0 9.696461243796197 1.5400106651309375
iteration information:  10000 11576 1.000058927362518 1.109847533649669e-05
iteration information:  20000 21576 1.0000287980091662 5.428451017801271e-06
iteration information:  30000 31576 1.0000190481747326 3.5919142359707857e-06
iteration information:  40000 41576 1.000014228357639 2.683622452019362e-06
iteration information:  50000 51576 1.0000113543246432 2.1418637093067053e-06
iteration information:  60000 61576 1.0000094458493414 1.7820445300081961e-06
iteration information:  70000 71576 1.000008086401247 1.52570048974852e-06
iteration information:  80000 81576 1.0000070688929852 1.3338123962089644e-06
iteration information:  90000 91576 1.0000062787448825 1.1847874791750144e-06
iteration information:  100000 101576 1.0000056474218038 1.065708333862019e-06
don

In [ ]:
def sigmoid(z):
    return 1/(1+np.exp(-z))